In [3]:
import time
import tweepy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
fbposts = pd.read_csv("./data/facebook-fact-check.csv")
gossipcop_fake = pd.read_csv("./data/gossipcop_fake.csv")
gossipcop_real = pd.read_csv("./data/gossipcop_real.csv")
politifact_fake = pd.read_csv("./data/politifact_fake.csv")
politifact_real = pd.read_csv("./data/politifact_real.csv")
media_bias = pd.read_csv("./data/MediaBiasChart.csv")

In [5]:
# tag = []
# for q in media_bias['Quality']:
#     if q <= 24.0:
#         tag.append('Fake')
#     elif q <= 32.0:
#         tag.append('Acceptable')
#     else:
#         tag.append('Real')
# media_bias['TagQuality'] = tag
# media_bias['TagQuality'].value_counts()

# fake_bias = media_bias.loc[media_bias['TagQuality'] == 'Fake']
# print(fake_bias['Source'].value_counts().head())

# nodes = media_bias['Source'].unique()
# origin = []
# dest = []
# weight = []

# for i in range(len(media_bias)):
#     origin.append(media_bias['Source'][i])
#     dest.append(media_bias['TagQuality'][i])
    
# graph = pd.DataFrame({'Source': origin, 'Target': dest})
# graph.rename(columns={0:"Id"}) #Adding an index column
# graph.index.name = "Id"
# graph.to_csv('edges.csv')

In [6]:
politifact_fake.head()

,id,news_url,title,tweet_ids
0,politifact15014,speedtalk.com/forum/viewtopic.php?t=51650,BREAKING: First NFL Team Declares Bankruptcy O...,937349434668498944\t937379378006282240\t937380...
1,politifact15156,politics2020.info/index.php/2018/03/13/court-o...,Court Orders Obama To Pay $400 Million In Rest...,972666281441878016\t972678396575559680\t972827...
2,politifact14745,www.nscdscamps.org/blog/category/parenting/467...,UPDATE: Second Roy Moore Accuser Works For Mic...,929405740732870656\t929439450400264192\t929439...
3,politifact14355,https://howafrica.com/oscar-pistorius-attempts...,Oscar Pistorius Attempts To Commit Suicide,886941526458347521\t887011300278194176\t887023...
4,politifact15371,http://washingtonsources.org/trump-votes-for-d...,Trump Votes For Death Penalty For Being Gay,915205698212040704\t915242076681506816\t915249...


In [7]:
import tweepy

consumer_key = '...'
consumer_secret = '...'
access_token = '...'
access_token_secret = '...'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

#Comprobar que funciona la api
status = api.get_status('937349434668498944')
print(status.user.id)
user = api.get_user(status.user.id)
print(user.screen_name)

4219197432
OfeliasHeaven


In [8]:
import time
import tweepy
import pandas as pd
import matplotlib.pyplot as plt

def get_tweet_status(tweet_id):
    '''
    Devuelve un objeto status con todos los datos de un tweet
    '''
    return api.get_status(tweet_id)

def get_user_by_id(id):
    '''
    Devuelve un objeto user a traves del id del user
    '''
    return api.get_user(id)
    
    
def get_screen_name_from_tweet(tweet_id):
    try:
        status = api.get_status(tweet_id)
        user = get_user_by_id(status.user.id)
        screen_name = user.screen_name
        return screen_name
    except tweepy.TweepError as e:
        return ('SuspendedAccount')    

def get_followers_page(screen_name):
    '''
    Devuelve los ids de los usuarios que sigue un user, screen_name es el nombre de usuario sin el '@'
    ej @NelsonMandela -> NelsonMandela
    '''
    ids = []
    for page in tweepy.Cursor(api.followers_ids, screen_name="OfeliasHeaven").pages():
        ids.extend(page)
        #time.sleep(60)
    return ids

def get_followers_limited(screen_name, total):
    '''
    Devuelve los ids de los usuarios que sigue un user, screen_name es el nombre de usuario sin el '@'
    ej @NelsonMandela -> NelsonMandela
    Obtiene solo un numero de ids de usuarios indicado en el parametro total
    '''
    ids = []
    for page in tweepy.Cursor(api.followers_ids, screen_name="OfeliasHeaven").items(total):
        ids.append(page)
    return ids

def get_followers_from_tweet_id(tweet_id, total = 0):
    '''
    Pasandole el id de un tweet obtenemos todos sus seguidores,
    se le puede pasar el parametro total y obtener solamente un numero concreto de ids
    '''
    user_id = get_tweet_status(tweet_id).user.id
    screen_name = get_user_by_id(user_id).screen_name
    if total == 0:
        followers_ids = get_followers_page(screen_name)
    else:
        followers_ids = get_followers_limited(screen_name,total)
    return followers_ids
    
    

def get_tweets(num_noticias):
    '''
    Le pasamos el numero de noticias de las cuales queremos obtener los tweets_ids
    Con el DataFrame de Pandas separamos los tweets de una misma noticia
    Devuelve un array de [titulos, array(tweets_ids)]
    '''
    df = pd.read_csv('./data/politifact_fake.csv') 
    #Esto se puede cambiar por otro dataset (ej: ./data/politifact_real.csv), pero de momento trabajamos con este
    df['tweet_ids'][0].split('\t')
    
    titulo = []
    tweets = []
    i = 0
    
    for tweet in df['tweet_ids']:
        if i < num_noticias:
            array_tweets = str(tweet).split('\t')
            #Se puede cambiar a que tenga al menos 5 o 10 tweets para probar
            if(len(array_tweets) > 1 and len(array_tweets) < 2000): #Solo si tiene algun tweet lo añadimos, 1 elemento es NaN
                tweets.append(array_tweets)
                titulo.append(df['title'][i])
                i = i+1
    noticias = [titulo, tweets]
    return noticias 

In [34]:
%%time
noticias = get_tweets(num_noticias=1)
total_tweets = 0
max_tweet = 0
number = 0
arrayPlot = np.zeros(10)
for i in range(len(noticias[1])):
    total_tweets = total_tweets + len(noticias[1][i])
    arrayPlot[len(noticias[1][i]) // 200] += 1
    if(max_tweet < len(noticias[1][i])):
        max_tweet = len(noticias[1][i])
        number = i                
print(total_tweets)
name_dict = {}

for i,v in enumerate(noticias[1]):
    for j, t in enumerate(v):
        #name = get_screen_name_from_tweet(v[j]) #Tarda mucho y se bloquea con el rate limit 
        if name != 'SuspendedAccount':
            if name in name_dict:
                name_dict[name] = name_dict[name] + 1
            else:
                name_dict[name] = 1
                
df = pd.DataFrame(list(name_dict.items()), columns=['name', 'count'])
df.rename(columns={0:"id"}) #Adding an index column
df.index.name = "id"
df.to_csv('./output/nodos.csv')

163
Wall time: 2min 6s
